In [1]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
from datetime import datetime
from operator import itemgetter
data_folder_path = 'C:\\Users\\goodluck\\Desktop\\DB'
raw_data_path =  "C:\\Users\\goodluck\\Desktop\\DB\\raw_data"
storage_path =  "C:\\Users\\goodluck\\Desktop\\DB\\database_storage"
daily_path =  "C:\\Users\\goodluck\\Desktop\\DB\\daily"
DTBS_path = os.path.join(storage_path, "DTBS.pkl")
DCBS_path = os.path.join(storage_path, "DCBS.pkl")

In [2]:
# re-open
with open(DCBS_path, 'rb') as f:  
    DCBS = pickle.load(f)
# re-open
with open(DTBS_path, 'rb') as f:  
    DTBS = pickle.load(f)

In [3]:
def select_condition(codes, dt, fcd):
    satisfy = []
    if fcd[0] == 'lp':
        lm = calculate_time(dt, 'p', 20)
        
        for code in codes:
            if low_prc(code, dt, lm, fcd[1]):
                satisfy.append(code)
        
    else:
        for code in codes:
            if DCBS[dt][code][fcd[0]] <= fcd[2] and  DCBS[dt][code][fcd[0]] >= fcd[1]:
                satisfy.append(code)
                    
    return satisfy


def takeSecond(elem):
    return elem[1]

def rank(codes, dt, idx, num, d):
    rk = []
    if num == -1:
        num = len(codes)
    for code in codes:
        if idx == 'lp':
            if not np.isnan(DTBS['A'][code][dt]['dp']):
                lm = calculate_time(dt, 'p', 20)
                prc = get_prcs(code, lm)
                lp = absolute_low(prc, 0)
                rk.append((code,  DCBS[dt][code]['dp']/lp))
            
            
        elif idx != 'dl':
            if not np.isnan( DCBS[dt][code][idx]):
                rk.append((code,  DCBS[dt][code][idx]))
        else:
            if not np.isnan( DCBS[dt][code][idx]):
                if  DCBS[dt][code][idx] != 999:
                    rk.append((code,  DCBS[dt][code][idx]))
    if num >= len(rk):
        num = len(rk)
        
     
#     reverse = True 降序
    if d == 'd':
        rk.sort(key=takeSecond, reverse = True)
    else:
        rk.sort(key=takeSecond, reverse = False)

    if idx == 'lp':
        lp = [i for i in rk if i[1] == 1.0]
        ot = []
        if num - len(lp) > 0:
            ot = rk[len(lp):num]
        
        low = lp + ot
        
    else:    
        low = rk[0:num]
#     print("In",dt , len(low), "of", len(codes),  "is selected")   

#     print(low)
    
    
    return [i[0] for i in low]
  

In [4]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
td


'2023-05-04'

In [9]:
select = list(DCBS[td].keys())

flt_cd = [['dp', 100, 130]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]

flt_cd = [['dp', 100, 130], ['bl',0, 3]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', 0.5, 9999]]
flt_cd = [['dp', 100, 130], ['bl',0, 7], ['trt', 2, 10], ['ytm', -1, 9999]]
for fcd in flt_cd:
    select = select_condition(select, td, fcd)
    
    
# rk_cd = [['lp', 10, 'a']]
rk_cd = [['dl', 600, 'a']]


rk_cd = [['ytm', 20, 'd']]
rk_cd = [['dl', 30, 'a']]
codes = rank(select, td, rk_cd[0][0], rk_cd[0][1], rk_cd[0][2])
len(codes)

30

In [10]:
for r in codes:
    print(r, DCBS[td][r]['cn'], DCBS[td][r]['sc'],  DCBS[td][r]['sn'], "收盘价：", DCBS[td][r]['dp'],  "转股溢价率：", DCBS[td][r]['cpr'], "到期收益率：", DCBS[td][r]['ytm'],"换手率：", DCBS[td][r]['trt'], '市值',  DCBS[td][r]['bl'], '双低', DCBS[td][r]['dl'])

118016.SH 京源转债 688096.SH 京源环保 收盘价： 118.697 转股溢价率： 20.87 到期收益率： 0.49 换手率： 6.28 市值 3.33 双低 139.567
123063.SZ 大禹转债 300021.SZ 大禹节水 收盘价： 126.953 转股溢价率： 15.01 到期收益率： -0.36 换手率： 3.11 市值 6.346 双低 141.963
123141.SZ 宏丰转债 300283.SZ 温州宏丰 收盘价： 121.536 转股溢价率： 22.67 到期收益率： -0.06 换手率： 3.28 市值 3.211 双低 144.20600000000002
113649.SH 丰山转债 603810.SH 丰山集团 收盘价： 124.018 转股溢价率： 20.36 到期收益率： -0.58 换手率： 2.8 市值 4.994 双低 144.378
123162.SZ 东杰转债 300486.SZ 东杰智能 收盘价： 124.948 转股溢价率： 19.61 到期收益率： -0.49 换手率： 2.75 市值 5.7 双低 144.558
113651.SH 松霖转债 603992.SH 松霖科技 收盘价： 119.5 转股溢价率： 26.2 到期收益率： -0.4 换手率： 4.34 市值 6.1 双低 145.7
113532.SH 海环转债 603817.SH 海峡环保 收盘价： 110.781 转股溢价率： 35.38 到期收益率： -0.47 换手率： 2.47 市值 4.585 双低 146.161
127054.SZ 双箭转债 002381.SZ 双箭股份 收盘价： 120.805 转股溢价率： 25.87 到期收益率： -0.72 换手率： 3.17 市值 5.136 双低 146.675
113591.SH 胜达转债 603687.SH 大胜达 收盘价： 122.925 转股溢价率： 23.92 到期收益率： -0.51 换手率： 2.2 市值 4.743 双低 146.845
123151.SZ 康医转债 300869.SZ 康泰医学 收盘价： 118.829 转股溢价率： 30.68 到期收益率： 0.46 换手率： 3.82 市值 6.999 双低 149.509
113628.SH 晨丰转债 

In [41]:
tdd =datetime.today()
td = tdd.strftime('%Y-%m-%d')
select = list(DCBS[td].keys())
for code in select:
    DCBS[td][code]['cat1'] = DTBS['B'][code]['cat1']
    DCBS[td][code]['cat2'] = DTBS['B'][code]['cat2']
    
# save
f_save = open(DCBS_path, 'wb')
pickle.dump(DCBS, f_save)
f_save.close()


In [34]:
DTBS['127082.SZ']

KeyError: '127082.SZ'